In [2]:
import math
import torch.nn as nn
import torch

In [3]:
"""
okay so here is the explanation which I understood
as we fintune the transformer block we instead of updation the
whole paramters we update some percentage of full paramters 
lets say our layer has weight matrix dimension (d_in,d_out)
now what happens is the authors of the paper found that 
generally in the weight matrix there are vectors which are not dependent 
therefore they are extra paramters which are not necessary to train so 
we can apply matrix decompostion where W =  A x B
where A = d_in X R and B = R X d_out therefore we add the matrix after training
"""
class LoRALayer(nn.Module):
    def __init__(self,in_dim, out_dim, rank, alpha):
        super().__init__()

        self.A = torch.nn.Parameter(torch.empty(in_dim,rank))
        nn.init.kaiming_uniform(self.A, a=math.sqrt(5))

        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha
    def forward(self, x):
        # print("A shape---->",self.A.shape)
        # print("B shape---->",self.B.shape)
        # print(x.shape)
        x = self.alpha * (x @ self.A @ self.B)
        return x     

In [4]:
class LinearWithLoRA(nn.Module):
    def __init__(self,linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(linear.in_features,linear.out_features, rank,alpha)
    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [6]:
def replace_linear_with_lora(model,rank,alpha):
    for name, module in model.named_children():
        if isinstance(module,torch.nn.Linear):
            setattr(model, name, LinearWithLoRA(module,rank,alpha))
        else:
            replace_linear_with_lora(module, rank, alpha)    

In [ ]:
"""
lets say we take an example of gpt-2-small(124M)
if we replace just all the linear layers with Lora 
linear layers 
we have linear layers in attention head
feed forward and linear ouptut layer 

multi head attention we have key value and quey matrix 
where for each matrix we have (d_in,d_out) so 3 x (d_in,d_out)
okay then the projection layer which is (d_out,d_out)

then we have ffn where we have 2 layers with relu activation
the ffn1 = (emb_dim,4 * emb_dim)
    ffn2 = (4 * emb_dim,emb_dim)

    and the last linear output layer (emb_dim,vocab_size)

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}
"gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12}

(3 * 768 * 768 + 768 * 768 + 768 * 4 * 768 * 2) * 12 + (768 * 50257) = 123,532,032
now we take r as 2 suppose 


"""